In [2]:
import torch as tc
import itertools as it
from typing import List

### Sample Gamma Distribution

In [3]:
# gamma(alpha = 1, beta = 1) is exp(\lambda = 1)
# sample from gamma (1, 1) is same as sample from exp(1)
# PDF(exp(1)) = 1 - e(-t)
# invPDF(exp(1)) = -ln(1-t) = -ln(t)
@tc.no_grad()
def sample_gamma_a_1(n: int, a: int) -> tc.FloatTensor:
	return -tc.rand(n, a, requires_grad=False).log().sum(dim = 1)
sample_gamma_a_1(10, 8)

tensor([ 7.5377,  6.0586,  8.5526, 13.3349,  9.5487,  9.8054,  7.2889,  8.9643,
        13.2315,  2.8670])

### Sample Beta Distribution

1. for independent gamma random variable x, y, we have PDF as $f(x,y)=\frac{1}{\Gamma(\alpha) \Gamma(\beta)} x^{\alpha - 1} y^ {\beta - 1} e^{-(x+y)}$
2. Have $ u = \frac{x}{x + y} $ and $ v = x + y$

In [4]:
# x ~ Beta (A, B) is same as
# x = a / (a + b) , where a ~ Gamma(A, 1), b ~ Gamma(B, 1)

@tc.no_grad()
def sample_beta_from_gamma(n: int, a: int, b: int) -> tc.FloatTensor:
	x = sample_gamma_a_1(n, a)
	y = sample_gamma_a_1(n, b)
	return x / (x + y)

sample_beta_from_gamma(10, 1, 10)

tensor([0.0287, 0.0060, 0.0792, 0.0999, 0.3192, 0.2688, 0.0229, 0.0521, 0.0711,
        0.0852])

### Sample dirichlet with 2 methods

In [5]:
# Faster sample, proof https://en.wikipedia.org/wiki/Dirichlet_distribution#From_gamma_distribution
@tc.no_grad()
def sample_dirichlet_from_gamma(n: int, alphas: List[int]) -> tc.FloatTensor:
	samples = tc.hstack([sample_gamma_a_1(n, a).unsqueeze(1) for a in alphas])
	return samples / samples.sum(dim = 1, keepdim = True)

sample_dirichlet_from_gamma(2, [1, 1, 1, 1])

tensor([[0.2218, 0.0781, 0.1213, 0.5788],
        [0.1225, 0.1701, 0.6672, 0.0402]])

In [6]:
# sample from marginal distribution over each dim one by one
# sample d - 1 times, and each time will "cut off" a segment
# from the total left over segment (left over segment length
# was initialized with 1)

@tc.no_grad()
def sample_dirichlet_from_beta(n: int, alphas: List[int]) -> tc.FloatTensor:
	total = tc.ones(n)
	psumalpha = list(it.accumulate(reversed(alphas)))
	psumalpha.pop()
	psumalpha.reverse()

	sampled = []
	for a, s in zip(alphas, psumalpha):
		x = total * sample_beta_from_gamma(n, a, s)
		sampled.append(x)
		total = total - x
	sampled.append(total)
	return tc.vstack(sampled).T

sample_dirichlet_from_beta(2, [1, 1, 1, 1])

tensor([[0.0469, 0.0304, 0.2721, 0.6506],
        [0.1282, 0.0337, 0.5022, 0.3359]])